In [ ]:
import os

In [ ]:
directory = "C:\\Users\\44070779\\Desktop\\docu"

In [ ]:
f_names = []
filenames = os.listdir(directory)
for file in filenames:
    f_name = os.path.join(directory, file)
    f_names.append(f_name)

In [ ]:
data_corpus = {}

for f in filenames:
    f_name = os.path.join(directory, f)
    fi = open(f_name)
    data = fi.read()
    data_corpus[f] = data

In [ ]:
def strip(raw_text):
    letters_only = re.sub('[\s+]', " ", raw_text)
    return letters_only

In [ ]:
import pandas as pd
import re

In [ ]:
data = pd.DataFrame.from_dict(data_corpus,orient='index')

In [ ]:
data = data.reset_index()
data.columns = ['file_name','text']

In [ ]:
data['strip']= data['text'].apply(strip)

In [ ]:
import nltk
import string

In [ ]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return s

data['text_punc'] = data['strip'].apply(remove_punctuation)

In [ ]:
def numbers(raw_text):
    letters_only = re.sub(r'\d+', '', raw_text)
    return letters_only

In [ ]:
data['num'] = data['text_punc'].apply(numbers)

In [ ]:
from nltk import pos_tag

In [ ]:
words = set(nltk.corpus.words.words())
def non_eng_text(raw_text):
    wor = raw_text.split()
    meaningful_words = [w for w in wor \
         if w.lower() in words or not w.isalpha()]
    return( " ".join( meaningful_words ))

In [ ]:
data['non_eng']= data['num'].apply(non_eng_text)

In [ ]:
data['lower'] = data['non_eng'].str.lower()

In [ ]:
def clean_text(raw_text):
    letters_only = re.sub("[^a-z]", " ", raw_text)
    words = letters_only.split()
    corp = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']
    stops = set(corp)
    meaningful_words = [w for w in words if not w in stops and len(w)>1]
    return( " ".join( meaningful_words ))

In [ ]:
data['stop']= data['lower'].apply(clean_text)

In [ ]:
from nltk import pos_tag

In [ ]:
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
def lemma(raw_text):
    wor = raw_text.split()
    wnl = WordNetLemmatizer()
    meaningful = [wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i,j in pos_tag(wor)]
    return ( " ".join( meaningful))

In [ ]:
data['text_lemma']= data['stop'].apply(lemma)

In [ ]:
corp = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=500000, stop_words = corp, use_idf= True, ngram_range=(1,3))
%time train_data_feature=tfidf_vectorizer.fit_transform(data['text_lemma'])
print(train_data_feature.shape)

In [ ]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
tf_vectorizer = CountVectorizer( max_features=500000, stop_words = corp)
tf = tf_vectorizer.fit_transform(data['text_lemma'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
import sklearn

In [ ]:
train_data_norm = sklearn.preprocessing.normalize(train_data_feature)
tf_norm = sklearn.preprocessing.normalize(tf)

In [ ]:
train_data_np = train_data_norm.toarray()
tf_np = tf_norm.toarray()

In [ ]:
sum(train_data_np[1])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(train_data_feature)


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
titles = open('NAME.txt').read().split('\n')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Latent Dirichlet Allocation implementation with Gensim

from gensim import corpora, models, similarities 

In [ ]:
#strip any proper names from a text...unfortunately right now this is yanking the first word from a sentence too.
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()


In [ ]:
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns


In [ ]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [ ]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


In [ ]:
#remove proper names
preprocess = [strip_proppers(doc) for doc in data['stop']]

%time tokenized_text = [tokenize_and_stem(text) for text in preprocess]

%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]


In [ ]:
texts = [[word for word in text] for text in texts]

dictionary = corpora.Dictionary(texts)

In [ ]:
dictionary

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
 lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)

In [ ]:
print(lda[corpus[0]])

In [ ]:
len(lda[corpus[0]])

In [ ]:
import numpy as np

In [ ]:
topics = lda.print_topics(5, num_words=20)

In [ ]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)

In [ ]:
topics_matrix_1 = np.array(topics_matrix, dtype=object)

In [ ]:
topics_matrix_1.shape

In [ ]:
topic_words = topics_matrix_1[:,1]

In [ ]:
 print(([str(word) for word in i]) for i in topic_words)

In [ ]:
mm = [dictionary.doc2bow(text) for text in texts]

In [ ]:
lda_corpus = lda[mm]

In [ ]:
from itertools import chain

In [ ]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))
threshold = sum(scores)/len(scores)
print threshold
print

In [ ]:
lda_corpus = [max(prob,key=lambda y:y[1]) for prob in lda[mm] ]
playlists = [[] for i in xrange(5)]
                        
for i, x in enumerate(lda_corpus):
    playlists[x[0]].append(data['file_name'][i])

In [ ]:
playlists